<a href="https://colab.research.google.com/github/chian0501/ChatTalker/blob/master/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 初始化

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install tqdm > log.txt
!pip install finlab > log.txt
!pip install talib-binary > log.txt
!pip install pandas > log.txt
!pip install numpy > log.txt
!pip install TA-Lib > log.txt
!pip install pandas_ta > log.txt

In [ ]:
# 導入必要的套件
import pandas as pd
import numpy as np
from finlab import data
import os
import logging
from datetime import datetime
import time
from tqdm import tqdm

In [ ]:
import finlab
finlab.login('6yYtEY2i1ze+WcqE60nCgC0NuN1Sbbk981t7B9FpCsR12KlgU1wL9154tVQVbcIK#vip_m')

輸入成功!


In [ ]:
# finlab_init.py

class FinLabInitializer:
    def __init__(self, api_key):
        """
        初始化 FinLab 環境
        Args:
            api_key (str): FinLab 的 API 金鑰
        """
        self.api_key = api_key
        self.initialized = False

    def setup(self):
        """設置並初始化 FinLab 環境"""
        try:
            # 掛載 Google Drive
            from google.colab import drive
            drive.mount('/content/drive')
            print("成功掛載 Google Drive")

            # 安裝必要套件
            self._install_requirements()

            # 導入必要套件
            self._import_packages()

            # 登入 FinLab
            self._login()

            self.initialized = True
            print("FinLab 環境初始化完成")

        except Exception as e:
            print(f"初始化過程發生錯誤: {str(e)}")
            raise

    def _install_requirements(self):
        """安裝必要的套件"""
        import subprocess

        packages = [
            'tqdm',
            'finlab',
            'talib-binary',
            'pandas',
            'numpy',
            'TA-Lib',
            'pandas_ta'
        ]

        for package in packages:
            subprocess.run(['pip', 'install', package],
                         capture_output=True,
                         text=True)

    def _import_packages(self):
        """導入必要的套件"""
        global pd, np, data, os, logging, datetime, time, tqdm

        import pandas as pd
        import numpy as np
        from finlab import data
        import os
        import logging
        from datetime import datetime
        import time
        from tqdm import tqdm

    def _login(self):
        """登入 FinLab"""
        import finlab
        finlab.login(self.api_key)

    def verify_initialization(self):
        """驗證初始化狀態"""
        if not self.initialized:
            return False

        # 測試數據獲取
        try:
            test_data = data.get('price:收盤價')
            return test_data is not None
        except:
            return False



In [ ]:
class TaiwanStockDatabase:
    def __init__(self):
        """初始化台股資料庫"""
        # 設定資料庫路徑
        self.db_path = '/content/drive/MyDrive/Colab Notebooks/finlab_db'
        self.excel_path = '/content/drive/MyDrive/Colab Notebooks/stock_db'

        # 先設定日誌系統
        self.setup_logging()

        # 再建立目錄
        self.create_directories()

        # 初始化資料容器
        self.company_info = None
        self.etf_info = None
        self.market_info = None
        self.stock_trading = None
        self.institutional = None
        self.fundamental = None

    def setup_logging(self):
        """設定日誌系統"""
        self.logger = logging.getLogger('TaiwanStockDB')
        self.logger.setLevel(logging.INFO)

        # 確保日誌目錄存在
        os.makedirs(self.db_path, exist_ok=True)

        # 建立檔案處理器
        fh = logging.FileHandler(f'{self.db_path}/stock_db.log')
        fh.setLevel(logging.INFO)

        # 建立控制台處理器
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)

        # 設定日誌格式
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)

        # 清除現有的處理器（避免重複）
        self.logger.handlers = []

        # 添加處理器
        self.logger.addHandler(fh)
        self.logger.addHandler(ch)

    def create_directories(self):
        """建立必要的目錄結構"""
        directories = [
            self.db_path,
            self.excel_path,
            f"{self.excel_path}/company_info",
            f"{self.excel_path}/market_data",
            f"{self.excel_path}/stock_data",
            f"{self.excel_path}/etf_data",
            f"{self.excel_path}/institutional_data",
            f"{self.excel_path}/fundamental_data"
        ]

        for directory in directories:
            os.makedirs(directory, exist_ok=True)
            self.logger.info(f"建立目錄: {directory}")




    def update_company_info(self):
        """更新公司基本資料"""
        try:
            self.company_info = {
                '公司基本資料': data.get('company_basic_info'),
                '主要經營業務': data.get('company_main_business')
            }

            # 儲存到Excel
            save_path = f"{self.excel_path}/company_info/company_basic_info.xlsx"
            with pd.ExcelWriter(save_path) as writer:
                self.company_info['公司基本資料'].to_excel(writer, sheet_name='基本資料')
                self.company_info['主要經營業務'].to_excel(writer, sheet_name='經營業務')

            self.logger.info("公司基本資料更新完成")
            return True
        except Exception as e:
            self.logger.error(f"更新公司基本資料時發生錯誤: {str(e)}")
            return False

    def update_etf_info(self):
        """更新ETF資訊"""
        try:
            securities = data.get('security_categories')
            etf_list = securities[securities['category'] == 'ETF']

            self.etf_info = {
                'ETF基本資訊': etf_list,
                'ETF價格': data.get('price:收盤價')[etf_list.index],
                'ETF成交量': data.get('price:成交股數')[etf_list.index]
            }

            # 儲存到Excel
            save_path = f"{self.excel_path}/etf_data/etf_info.xlsx"
            with pd.ExcelWriter(save_path) as writer:
                self.etf_info['ETF基本資訊'].to_excel(writer, sheet_name='ETF清單')
                self.etf_info['ETF價格'].to_excel(writer, sheet_name='價格資料')
                self.etf_info['ETF成交量'].to_excel(writer, sheet_name='成交量資料')

            self.logger.info("ETF資訊更新完成")
            return True
        except Exception as e:
            self.logger.error(f"更新ETF資訊時發生錯誤: {str(e)}")
            return False

    def update_market_info(self):
        """更新大盤資訊"""
        try:
            self.market_info = {
                '大盤指數': {
                    '發行量加權股價指數': {
                        '開盤指數': data.get('taiex_total_index:開盤指數'),
                        '最高指數': data.get('taiex_total_index:最高指數'),
                        '最低指數': data.get('taiex_total_index:最低指數'),
                        '收盤指數': data.get('taiex_total_index:收盤指數')
                    },
                    '報酬指數': data.get('benchmark_return:發行量加權股價報酬指數')
                },
                '市場成交統計': {
                    '成交股數': data.get('market_transaction_info:成交股數'),
                    '成交金額': data.get('market_transaction_info:成交金額'),
                    '成交筆數': data.get('market_transaction_info:成交筆數')
                }
            }

            # 儲存到Excel
            save_path = f"{self.excel_path}/market_data/market_info.xlsx"
            with pd.ExcelWriter(save_path) as writer:
                for key, value in self.market_info['大盤指數']['發行量加權股價指數'].items():
                    value.to_excel(writer, sheet_name=f'大盤_{key}')
                self.market_info['大盤指數']['報酬指數'].to_excel(writer, sheet_name='報酬指數')

            self.logger.info("大盤資訊更新完成")
            return True
        except Exception as e:
            self.logger.error(f"更新大盤資訊時發生錯誤: {str(e)}")
            return False

    def update_stock_trading(self):
        """更新個股交易資訊"""
        try:
            self.stock_trading = {
                '價格資訊': {
                    '收盤價': data.get('price:收盤價'),
                    '開盤價': data.get('price:開盤價'),
                    '最高價': data.get('price:最高價'),
                    '最低價': data.get('price:最低價'),
                    '成交股數': data.get('price:成交股數'),
                    '成交金額': data.get('price:成交金額'),
                    '成交筆數': data.get('price:成交筆數')
                },
                '還原權值': {
                    '還原收盤價': data.get('etl:adj_close'),
                    '還原開盤價': data.get('etl:adj_open'),
                    '還原最高價': data.get('etl:adj_high'),
                    '還原最低價': data.get('etl:adj_low')
                }
            }

            # 儲存到Excel (分批儲存以避免檔案過大)
            for category, data_dict in self.stock_trading.items():
                save_path = f"{self.excel_path}/stock_data/{category}.xlsx"
                with pd.ExcelWriter(save_path) as writer:
                    for name, df in data_dict.items():
                        df.to_excel(writer, sheet_name=name)

            self.logger.info("個股交易資訊更新完成")
            return True
        except Exception as e:
            self.logger.error(f"更新個股交易資訊時發生錯誤: {str(e)}")
            return False

    def update_institutional_data(self):
        """更新籌碼面資料"""
        try:
            self.institutional = {
                '三大法人': {
                    '外資': {
                        '買進股數': data.get('institutional_investors_trading_summary:外陸資買進股數(不含外資自營商)'),
                        '賣出股數': data.get('institutional_investors_trading_summary:外陸資賣出股數(不含外資自營商)'),
                        '買賣超股數': data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
                    },
                    '投信': {
                        '買進股數': data.get('institutional_investors_trading_summary:投信買進股數'),
                        '賣出股數': data.get('institutional_investors_trading_summary:投信賣出股數'),
                        '買賣超股數': data.get('institutional_investors_trading_summary:投信買賣超股數')
                    },
                    '自營商': {
                        '自行買賣': data.get('institutional_investors_trading_summary:自營商買賣超股數(自行買賣)'),
                        '避險': data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)')
                    }
                },
                '股東結構': {
                    '外資持股': {
                        '持股數': data.get('foreign_investors_shareholding:全體外資及陸資持有股數'),
                        '持股比率': data.get('foreign_investors_shareholding:全體外資及陸資持股比率')
                    },
                    '董監持股': {
                        '持股數': data.get('internal_equity_changes:董監持有股數'),
                        '持股比率': data.get('internal_equity_changes:董監持有股數占比')
                    }
                }
            }

            # 儲存到Excel
            save_path = f"{self.excel_path}/institutional_data/institutional_info.xlsx"
            with pd.ExcelWriter(save_path) as writer:
                for category in ['三大法人', '股東結構']:
                    for subcategory, data_dict in self.institutional[category].items():
                        for name, df in data_dict.items():
                            sheet_name = f"{category}_{subcategory}_{name}"[:31]  # Excel工作表名稱限制
                            df.to_excel(writer, sheet_name=sheet_name)

            self.logger.info("籌碼面資料更新完成")
            return True
        except Exception as e:
            self.logger.error(f"更新籌碼面資料時發生錯誤: {str(e)}")
            return False

    def update_fundamental_data(self):
        """更新基本面資料"""
        try:
            self.fundamental = {
                '月營收': {
                    '當月營收': data.get('monthly_revenue:當月營收'),
                    '上月營收': data.get('monthly_revenue:上月營收'),
                    '去年當月營收': data.get('monthly_revenue:去年當月營收'),
                    '上月比較增減': data.get('monthly_revenue:上月比較增減(%)'),
                    '去年同月增減': data.get('monthly_revenue:去年同月增減(%)')
                },
                '財務指標': {
                    '營業利益': data.get('fundamental_features:營業利益'),
                    'EBITDA': data.get('fundamental_features:EBITDA'),
                    '營運現金流': data.get('fundamental_features:營運現金流'),
                    'ROE': data.get('fundamental_features:ROE稅後'),
                    'ROA': data.get('fundamental_features:ROA稅後息前'),
                    '營業毛利率': data.get('fundamental_features:營業毛利率'),
                    '營業利益率': data.get('fundamental_features:營業利益率'),
                    '稅後淨利率': data.get('fundamental_features:稅後淨利率')
                },
                '本益比、殖利率': {
                    '本益比': data.get('price_earning_ratio:本益比'),
                    '殖利率': data.get('price_earning_ratio:殖利率(%)'),
                    '股價淨值比': data.get('price_earning_ratio:股價淨值比')
                }
            }

            # 儲存到Excel
            save_path = f"{self.excel_path}/fundamental_data/fundamental_info.xlsx"
            with pd.ExcelWriter(save_path) as writer:
                for category, data_dict in self.fundamental.items():
                    for name, df in data_dict.items():
                        sheet_name = f"{category}_{name}"[:31]
                        df.to_excel(writer, sheet_name=sheet_name)

            self.logger.info("基本面資料更新完成")
            return True
        except Exception as e:
            self.logger.error(f"更新基本面資料時發生錯誤: {str(e)}")
            return False

    def update_all_data(self):
        """更新所有資料"""
        self.logger.info("開始更新所有資料...")

        update_functions = [
            self.update_company_info,
            self.update_etf_info,
            self.update_market_info,
            self.update_stock_trading,
            self.update_institutional_data,
            self.update_fundamental_data
        ]

        results = []
        for func in tqdm(update_functions, desc="更新進度"):
            results.append(func())

        success_count = sum(results)
        self.logger.info(f"資料更新完成: {success_count}/{len(update_functions)} 個項目成功")

    def get_stock_summary(self, stock_id):
        """獲取特定股票的綜合資訊"""
        try:
            summary = {
                '基本資料': self.company_info['公司基本資料'].loc[stock_id].to_dict(),
                '今日交易': {
                    k: v.loc[stock_id].iloc[-1]
                    for k, v in self.stock_trading['價格資訊'].items()
                },
                '籌碼面': {
                    '三大法人買賣超': {
                        '外資': self.institutional['三大法人']['外資']['買賣超股數'].loc[stock_id].iloc[-1],
                        '投信': self.institutional['三大法人']['投信']['買賣超股數'].loc[stock_id].iloc[-1],
                        '自營商': self.institutional['三大法人']['自營商']['自行買賣'].loc[stock_id].iloc[-1]
                    }
                },
                '基本面': {
                    '月營收': self.fundamental['月營收']['當月營收'].loc[stock_id].iloc[-1],
                    'ROE': self.fundamental['財務指標']['ROE'].loc[stock_id].iloc[-1],
                    '本益比': self.fundamental['本益比、殖利率']['本益比'].loc[stock_id].iloc[-1]
                }
            }
            return summary
        except Exception as e:
            self.logger.error(f"獲取股票 {stock_id} 資訊時發生錯誤: {str(e)}")
            return None


In [ ]:
def test_taiwan_stock_db():
    """測試台股資料庫功能"""
    try:
        # 初始化資料庫
        print("開始測試台股資料庫...")
        db = TaiwanStockDatabase()

        # 測試更新所有資料
        print("\n1. 測試資料更新功能")
        db.update_all_data()

        # 測試獲取特定股票資訊
        print("\n2. 測試股票資訊獲取功能")
        test_stocks = ['2330', '2317', '2454']  # 測試股票：台積電、鴻海、聯發科

        for stock_id in test_stocks:
            print(f"\n獲取 {stock_id} 的資訊：")
            stock_info = db.get_stock_summary(stock_id)
            if stock_info:
                print(f"基本資料：{stock_info['基本資料']['公司名稱']}")
                print(f"今日收盤價：{stock_info['今日交易']['收盤價']}")
                print(f"三大法人買賣超：")
                print(f"  外資：{stock_info['籌碼面']['三大法人買賣超']['外資']}")
                print(f"  投信：{stock_info['籌碼面']['三大法人買賣超']['投信']}")
                print(f"基本面：")
                print(f"  月營收：{stock_info['基本面']['月營收']}")
                print(f"  ROE：{stock_info['基本面']['ROE']}")

        # 測試資料存取路徑
        print("\n3. 測試資料存取路徑")
        print(f"資料庫路徑: {db.db_path}")
        print(f"Excel檔案路徑: {db.excel_path}")

        # 測試日誌功能
        print("\n4. 測試日誌功能")
        db.logger.info("這是一條測試日誌")
        print("請檢查日誌檔案是否正確記錄")

        print("\n測試完成！")

    except Exception as e:
        print(f"測試過程中發生錯誤: {str(e)}")
        raise

# 執行測試
if __name__ == "__main__":
    # main.py
    from finlab_init import FinLabInitializer

    # 創建初始化器實例
    API_KEY = '6yYtEY2i1ze+WcqE60nCgC0NuN1Sbbk981t7B9FpCsR12KlgU1wL9154tVQVbcIK#vip_m'
    finlab_init = FinLabInitializer(API_KEY)

    # 執行初始化
    finlab_init.setup()

    # 驗證初始化是否成功
    if finlab_init.verify_initialization():
        print("環境初始化成功，可以開始使用 FinLab 功能")
    else:
        print("環境初始化失敗，請檢查設置")


    # 執行測試
    test_taiwan_stock_db()


ModuleNotFoundError: No module named 'finlab_init'